## Growth Curve Model (GCM)

This is a Jupyter Notebook that can be opened in Google Ciolab (if the underlying link as specified in the config.yml exists!).

It can do many things, like displaying and executing code:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from semopy import Model, ModelMeans, semplot, calc_stats
from semopy import ModelMeans
from semopy.means import estimate_means

In [ ]:
# Read the CSV file using a relative path
anx = pd.read_csv("../LSCM_and_GCM/Datasets/anx.csv")
#anx = anx - anx.mean()

#anx.head()

# Define the growth curve model
GCM1 = '''
Iw =~ 1.0*t1 + 1.0*t2 + 1.0*t3 + 1.0*t4
Sw =~ 0*t1 + 1.0*t2 + 2.0*t3 + 3.0*t4

t1 ~ 0*1
t2 ~ 0*1
t3 ~ 0*1
t4 ~ 0*1

'''

# Create and fit the model
model = ModelMeans(GCM1)

# # Fit the model to the data
results = model.fit(anx)
# print(results)

# # Print the standardized estimates
estimates = (model.inspect())
print(estimates)


# # Print the fit measures
fit_measures = calc_stats(model).T
print(fit_measures)

print(estimate_means(model))


# Plot the path diagram inline
semplot(model, plot_covs = True, filename="../LSCM_and_GCM/Datasets/GCM.pdf") #Need graphViz installed

latent_means = model.predict_factors(anx).mean()
print(latent_means)



   lval  op rval  Estimate  Std. Err    z-value   p-value
0    t1   ~   Iw  1.000000         -          -         -
1    t1   ~   Sw  0.000000         -          -         -
2    t2   ~   Iw  1.000000         -          -         -
3    t2   ~   Sw  1.000000         -          -         -
4    t3   ~   Iw  1.000000         -          -         -
5    t3   ~   Sw  2.000000         -          -         -
6    t4   ~   Iw  1.000000         -          -         -
7    t4   ~   Sw  3.000000         -          -         -
8    Iw  ~~   Iw  0.068442   0.01018   6.723346       0.0
9    Sw  ~~   Sw  0.046316  0.004485   10.32739       0.0
10   Sw  ~~   Iw -0.007780  0.005178  -1.502479  0.132973
11   t1  ~~   t1  0.096048  0.010925   8.791251       0.0
12   t2  ~~   t2  0.093066   0.00742  12.542855       0.0
13   t3  ~~   t3  0.085799  0.008474  10.124916       0.0
14   t4  ~~   t4  0.082242  0.015285   5.380681       0.0
                   Value
DoF             3.000000
DoF Baseline    6.0000

In [65]:
from semopy import Model
import pandas as pd

anx = pd.read_csv("../LSCM_and_GCM/Datasets/anx.csv")
# Define the growth curve model
GCM1 = '''
Iw =~ 1*t1 + 1*t2 + 1*t3 + 1*t4
Sw =~ 0*t1 + 1*t2 + 2*t3 + 3*t4

# Fix the intercept of t1 to 0 to identify the model
t1 ~ 0
'''

# Fit the model
model = Model(GCM1)
results = model.fit(anx)

# Extract the parameter estimates
estimates = model.inspect()

# Show the intercepts for observed variables and latent factors
print(estimates[estimates['op'] == '~1'])  # Intercepts for observed variables
print(estimates[estimates['op'] == '=~'])  # Loadings for latent factors

# To manually compute the latent intercepts, you'd use the factor loadings and observed means


KeyError: 'Variables 0 are missing from data.'

In [44]:
from semopy.examples import \
     multivariate_regression
from semopy.means import estimate_means
from semopy import Model
desc = multivariate_regression.get_model()
data = multivariate_regression.get_data()

model = Model(desc)
model.fit(data)
print(estimate_means(model))
print(desc)

  lval op rval  Estimate
0   x1  ~    1 -0.043750
1   x2  ~    1  0.073704
2   x3  ~    1 -0.056712
3   y1  ~    1 -1.456624
4   y2  ~    1  0.929309
5   y3  ~    1  0.992039
y1, y2, y3 ~ x1 + x2 + x3
